In [42]:
# Import necessary libraries
import pandas as pd
import json

# Load the CSV dataset
df = pd.read_csv("semi_strut.csv")

# Display the first few rows of the dataset
df.head()

,Document ID,Content
0,1,"{\r\n ""title"": ""Introduction to Python"",\r\n..."
1,2,"{\r\n ""title"": ""Data Analysis with Pandas"",\..."
2,3,"{\r\n ""title"": ""Web Development with Flask"",..."
3,4,"{\r\n ""title"": ""Machine Learning with Scikit..."
4,5,"{\r\n ""title"": ""Data Visualization with Matp..."


In [76]:
# Tokenization function to extract terms from the JSON-like content
def tokenize_content(content):
    content_dict = json.loads(content)
    terms = []

    # Remember to extract both 
    # 1 .Extract terms from various fields (title, keywords, and content)
    terms.extend(content_dict.get("title","").split())
    terms.extend(content_dict.get("author","").split())
    terms.extend(content_dict.get("keywords",[]))
    
    # 2. Extract terms from sections' titles and content
    sections = content_dict.get("sections",[])
    for section in sections:
        terms.extend(section.get("title","").split())
        terms.extend(section.get("content","").split()) 

    return terms
# 3. Tokenize the content and create a new column "Terms"
df["Term"] = df["Content"].apply(tokenize_content)
# 4. Implement a preprocessing function that converts terms to lowercase, removes punctuation, and removes common stop words.
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
def preprocess_terms(terms):
    terms = [term.lower() for term in terms if term.isalnum()]
    # Create another new column "Terms_preprocessed"
    return terms
# 5. you can display the DataFrame
df[["Document ID","Term"]]
# def tokenize_content(content):

,Document ID,Term
0,1,"[Introduction, to, Python, John, Doe, Python, ..."
1,2,"[Data, Analysis, with, Pandas, Jane, Smith, Py..."
2,3,"[Web, Development, with, Flask, Mike, Johnson,..."
3,4,"[Machine, Learning, with, Scikit-Learn, Emily,..."
4,5,"[Data, Visualization, with, Matplotlib, Robert..."


In [40]:
#

In [5]:
# Initialize an empty inverted index dictionary
# Build the inverted index

In [7]:
# Initialize an empty inverted index dictionary the extracted terms 

In [8]:
# perform boolean operations on postings lists for Boolean search operations
# 1. "Python" OR "Pandas"
# 2. "Python" AND "data"

In [41]:
import pandas as pd
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Sample data
data = {
    "Document ID": [1, 2, 3, 4, 5],
    "Content": [
        {
            "title": "Introduction to Python",
            "author": "John Doe",
            "date": "2023-01-15",
            "keywords": ["Python", "programming", "beginner"],
            "sections": [
                {
                    "title": "Getting Started",
                    "content": "Python is a versatile programming language..."
                },
                {
                    "title": "Basic Syntax",
                    "content": "Python syntax is easy to understand..."
                }
            ]
        },
        {
            "title": "Data Analysis with Pandas",
            "author": "Jane Smith",
            "date": "2023-02-20",
            "keywords": ["Python", "Pandas", "data analysis"],
            "sections": [
                {
                    "title": "Introduction to Pandas",
                    "content": "Pandas is a popular library for data analysis..."
                },
                {
                    "title": "DataFrames",
                    "content": "DataFrames are a core data structure in Pandas..."
                }
            ]
        },
        {
            # ... (other documents)
        }
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Initialize NLTK resources
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# Function to extract terms from a JSON-like content
def tokenize_content(content):
    # Initialize the list to store extracted terms
    terms = []
    extract_terms = []
    # Extract terms from various fields (title, keywords, and content)
    title_terms = extract_terms(content["title"])
    keyword_terms = [term.lower() for term in content["keywords"]]
    
    # Extract terms from content sections
    content_terms = []
    for section in content["sections"]:
        content_terms.extend(extract_terms(section["title"]))
        content_terms.extend(extract_terms(section["content"]))
    
    # Combine all the terms for this content
    terms.extend(title_terms)
    terms.extend(keyword_terms)
    terms.extend(content_terms)
    
    return terms

# Function to preprocess terms
def preprocess_terms(terms):
    # Convert terms to lowercase, remove punctuation, and remove stopwords
    terms = [term.lower() for term in terms if term.isalnum()]
    terms = [stemmer.stem(term) for term in terms if term not in stop_words]
    
    return terms

# Apply tokenization and preprocessing to the Content column
df["Terms"] = df["Content"].apply(tokenize_content)
df["Terms_preprocessed"] = df["Terms"].apply(preprocess_terms)

# Display the DataFrame
print(df)


ValueError: All arrays must be of the same length